In [1]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [2]:
from openai import OpenAI

client = OpenAI(
    api_key=os.getenv("LLM_API_KEY"), 
    base_url=os.getenv("LLM_BASE_URL"),
)

In [3]:
response = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "What is the colour of a rose?"}
    ],
    temperature=0.1,
    model=os.getenv("LLM_MODEL_NAME")
)

In [6]:
raw_response = response.choices[0].message.content
print(raw_response)

<think>
Okay, so I need to figure out the color of a rose. Hmm, I know that roses come in different colors, but I'm not entirely sure about all the possibilities. Let me think about this step by step.

First, the most common color I associate with roses is red. I've seen red roses in bouquets, in gardens, and they're often used for Valentine's Day or to express love. So red is definitely a color roses can be.

But wait, I've also seen other colors. There are pink roses, right? I think pink roses are pretty common too. They might symbolize different things, like friendship or appreciation. So pink is another color.

What about white roses? I remember seeing white roses at weddings. They might symbolize purity or innocence. So white is another color.

I think there are also yellow roses. I've seen those in some arrangements. They might represent happiness or celebration. So yellow is another color.

Are there more colors? I'm trying to recall. I think I've heard of orange roses before. T

In [7]:
def format_response(raw_response):
    # separate the answer and the thought process
    parts = raw_response.split("</think>")

    assert len(parts) == 2

    thought = parts[0]
    # remove the remaining think tag
    thought = thought.replace("<think>", "").strip()

    answer = parts[1]
    # remove if there's an "Answer:" part
    answer = answer.replace("Answer:", "").strip()


    return thought, answer

In [8]:
format_response(raw_response)

("Okay, so I need to figure out the color of a rose. Hmm, I know that roses come in different colors, but I'm not entirely sure about all the possibilities. Let me think about this step by step.\n\nFirst, the most common color I associate with roses is red. I've seen red roses in bouquets, in gardens, and they're often used for Valentine's Day or to express love. So red is definitely a color roses can be.\n\nBut wait, I've also seen other colors. There are pink roses, right? I think pink roses are pretty common too. They might symbolize different things, like friendship or appreciation. So pink is another color.\n\nWhat about white roses? I remember seeing white roses at weddings. They might symbolize purity or innocence. So white is another color.\n\nI think there are also yellow roses. I've seen those in some arrangements. They might represent happiness or celebration. So yellow is another color.\n\nAre there more colors? I'm trying to recall. I think I've heard of orange roses befor

In [14]:
class LLMEngine:
    def __init__(self, api_key, model, base_url, system_input):
        self.api_key = api_key
        self.base_url = base_url
        self.model = model
        self.system_input = system_input
        
    def __prepare_inference_input(self, user_input):
        messages = []
        if self.system_input:
            messages.append(
                {"role": "system", "content": self.system_input}
            )
        if user_input:
            messages.append(
                {"role": "user", "content": user_input}
            )
            
        return messages
        
    def __generate_raw_response(self, user_input, temperature=0.1):
        messages = self.__prepare_inference_input(user_input)
        assert messages
        
        client = OpenAI(
            api_key=self.api_key, 
            base_url=self.base_url,
        )
        
        response = client.chat.completions.create(
            messages = messages,
            temperature = temperature,
            model = self.model
        )
        
        return response.choices[0].message.content
    
    def generate(self, user_input):
        raw_response = self.__generate_raw_response(user_input)
        formatted = format_response(raw_response)
        return formatted
        
    def format_response(self, raw_response):
        pass
        

In [15]:
class DeepSeekR1Engine(LLMEngine):
    def __init__(self, api_key, model, base_url, system_input):
        super().__init__(api_key, model, base_url, system_input)
        
    def format_response(raw_response):
        # separate the answer and the thought process
        parts = raw_response.split("</think>")

        assert len(parts) == 2

        thought = parts[0]
        # remove the remaining think tag
        thought = thought.replace("<think>", "").strip()

        answer = parts[1]
        # remove if there's an "Answer:" part
        answer = answer.replace("Answer:", "").strip()


        return thought, answer
    

In [16]:
r1_engine = DeepSeekR1Engine(
    api_key=os.getenv("LLM_API_KEY"),
    base_url=os.getenv("LLM_BASE_URL"),
    model=os.getenv("LLM_MODEL_NAME"),
    system_input=None
)

In [17]:
r1_engine.generate("what is the colour of a rose?")

("Okay, so I need to figure out the color of a rose. Hmm, roses are flowers, right? I remember seeing them in gardens and in bouquets. But wait, what color are they exactly? I think they come in different colors, but I'm not entirely sure. Let me try to recall.\n\nI know that red roses are really common, especially on Valentine's Day. People give red roses as a symbol of love. But are all roses red? I don't think so. I've seen pink roses too, maybe even white ones. Oh, and I think there are yellow roses as well. So, does that mean roses can be multiple colors?\n\nWait, maybe I should think about why roses have different colors. Is it just for looks, or does each color have a different meaning? I remember hearing that different colored roses can symbolize different things. Like red for love, pink for appreciation or friendship, white for purity or weddings, and yellow for happiness or celebration. So, if that's the case, then roses aren't just one color; they come in various colors depe